In [18]:
import pandas as pd
import numpy as np
import os
import re
import csv
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tfidf
from nltk.tokenize import word_tokenize
import nltk
import string

In [3]:
path='C:/Users/mom1/Desktop/571/'

In [4]:
names=["ID","title","IMDB URL","genre","director","cast","IMDB rating","mpaa","writer",
    "certificates","runtimes","year","plot summary","plot keywords","type",
    "full screen image URL"]
test=pd.read_table('C:/Users/mom1/Desktop/571/TestSet.txt',sep='|',header= None,names=names)

In [5]:
print test.shape
test.info()

(8737, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8737 entries, 0 to 8736
Data columns (total 16 columns):
ID                       8018 non-null float64
title                    8018 non-null object
IMDB URL                 8737 non-null object
genre                    7079 non-null object
director                 5348 non-null object
cast                     7074 non-null object
IMDB rating              3007 non-null float64
mpaa                     92 non-null object
writer                   4051 non-null object
certificates             1618 non-null object
runtimes                 3708 non-null object
year                     7917 non-null float64
plot summary             2538 non-null object
plot keywords            2583 non-null object
type                     2 non-null object
full screen image URL    2052 non-null object
dtypes: float64(3), object(13)
memory usage: 1.1+ MB


In [7]:
len(test.genre.unique())

670

In [8]:
test.head()

,ID,title,IMDB URL,genre,director,cast,IMDB rating,mpaa,writer,certificates,runtimes,year,plot summary,plot keywords,type,full screen image URL
0,NaN,NaN,http://akas.imdb.com/title/tt1000002/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,http://akas.imdb.com/title/tt1000026/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,http://akas.imdb.com/title/tt1000027/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,http://akas.imdb.com/title/tt1000028/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,http://akas.imdb.com/title/tt1000029/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Missing value percentages

In [11]:
quantitive_columns = test.select_dtypes(include=[np.number]).columns
print len(quantitive_columns)
miss_quant_null=test[quantitive_columns].isnull().sum(axis=0)/test.shape[0]
miss_quant_null.sort_values(ascending=False)

3


IMDB rating    0.655832
year           0.093854
ID             0.082294
dtype: float64

In [12]:
qualitative_columns = test.select_dtypes(include=[object]).columns
print len(qualitative_columns)
miss_qual=test[qualitative_columns].isnull().sum(axis=0)/test.shape[0]
miss_qual.sort_values(ascending=False)

13


type                     0.999771
mpaa                     0.989470
certificates             0.814811
full screen image URL    0.765137
plot summary             0.709511
plot keywords            0.704361
runtimes                 0.575598
writer                   0.536340
director                 0.387891
cast                     0.190340
genre                    0.189768
title                    0.082294
IMDB URL                 0.000000
dtype: float64

In [13]:
print test.genre.isnull().value_counts()
test=test.dropna(subset=["genre"])
test.genre.isnull().value_counts()

False    7079
True     1658
Name: genre, dtype: int64


False    7079
Name: genre, dtype: int64

In [16]:
columns=['ID', 'title','genre', 'director', 'cast',
       'writer','plot summary', 'plot keywords']
test1=test[columns]

Changing to lowercase, replacing "," and "u'" with space

In [19]:
pattern = re.compile(",")
cols=['title','genre','director','writer','cast','plot summary','plot keywords']
for i in cols:
    test1[i]=test1[i].str.lower()
    test1[i]=test1[i].replace(pattern," ")

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
pat=re.compile('(u\')')
test1["plot summary"]=test1["plot summary"].replace(pat," ")
test1["plot keywords"]=test1["plot keywords"].replace(pat," ")

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [21]:
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...


Add puntuations to stop words

In [22]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
stop_words.update(['!','u', '"' ,'#' ,'$' , '%' , '\'' , '(' , ')' , '*' , '+' , ',' , '-' ,'' ',' '',
                   '.' , '\\' , '/' , ':' , ';' ,'<' , '=' , '>' , '?' , '[' , ']' , '^' , '_', '`' , '~' ,'{', '}'  , '|' ])

Tokenize, remove stop words and then stem "title"

In [30]:
test1['title_nsw'] = np.empty((len(test1), 0)).tolist()
test1['title_stem'] = np.empty((len(test1), 0)).tolist()
test1["title_tok"] = [word_tokenize(i) for i in test1["title"] if pd.isnull(i)==False]
for index,row in test1.iterrows():
    if pd.isnull(row.title)==False:
        [test1.loc[index,"title_nsw"].append(w) for w in row["title_tok"] if not w in stop_words]
        for w in row["title_nsw"]:
            test1.loc[index,"title_stem"].append(str(ps.stem(w)))
    else:
        pass

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [31]:
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],"[the, bee]"
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]","[mickey, makes, money]"
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]","[episode, dated, 6, march, 2007]"
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]","[episode, dated, 10, march, 2007]"
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]","[i, fuckin, ', miss, cory, and, trevor]"


Tokenize and remove stop words "director"

In [47]:
test1.director=test1.director.astype(unicode,raise_on_error=False)

In [49]:
test1['director_nsw'] = np.empty((len(test1), 0)).tolist()
test1["director_tok"] = np.empty((len(test1), 0)).tolist()
for index,row in test1.iterrows():
    if pd.isnull(row.director)==False:
        row.director = unicode(row.director, errors='ignore')
        test1.loc[index,"director_tok"] = test1.loc[index,"director_tok"].append(word_tokenize(row.director))
        [test1.loc[index,"director_nsw"].append(w) for l in row["director_tok"] for w in l if not w in stop_words]
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,writer_nsw,director_tok,writer_tok,cast_nsw,cast_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],"[the, bee]","[louis, h., tolhurst]","[louis, h., tolhurst]",None,None,"[louis, h., tolhurst]",None
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]","[mickey, makes, money]","[micky, dolenz]","[louis, h., tolhurst]",None,None,"[louis, h., tolhurst]",None
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]","[episode, dated, 6, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...","[louis, h., tolhurst]",None,None,"[louis, h., tolhurst]",None
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]","[episode, dated, 10, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...","[louis, h., tolhurst]",None,None,"[louis, h., tolhurst]",None
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]","[i, fuckin, ', miss, cory, and, trevor]","[mike, clattenburg]","[louis, h., tolhurst]",None,None,"[louis, h., tolhurst]",None


Tokenize and remove stop words from "writer"

In [50]:
test1['writer_nsw'] = np.empty((len(test1), 0)).tolist()
test1["writer_tok"] = np.empty((len(test1), 0)).tolist()
for index,row in test1.iterrows():
    if pd.isnull(row.writer)==False:
        test1.loc[index,"writer_tok"] = test1.loc[index,"writer_tok"].append(word_tokenize(row.writer))
        [test1.loc[index,"writer_nsw"].append(w) for l in row["writer_tok"] for w in l if not w in stop_words]
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,writer_nsw,director_tok,writer_tok,cast_nsw,cast_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],"[the, bee]","[louis, h., tolhurst]",[],None,[],"[louis, h., tolhurst]",None
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]","[mickey, makes, money]","[micky, dolenz]","[colin, bostock-smith]",None,None,"[louis, h., tolhurst]",None
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]","[episode, dated, 6, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[louis, h., tolhurst]",None
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]","[episode, dated, 10, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[louis, h., tolhurst]",None
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]","[i, fuckin, ', miss, cory, and, trevor]","[mike, clattenburg]","[mike, clattenburg, mike, clattenburg, timm, h...",None,None,"[louis, h., tolhurst]",None


Tokenize and remove stop words from "cast"

In [52]:
test1['cast_nsw'] = np.empty((len(test1), 0)).tolist()
test1["cast_tok"] = np.empty((len(test1), 0)).tolist()
for index,row in test1.iterrows():
    if pd.isnull(row.cast)==False:
        row.cast = unicode(row.cast, errors='ignore')
        test1.loc[index,"cast_tok"] = test1.loc[index,"cast_tok"].append(word_tokenize(row.cast))
        [test1.loc[index,"cast_nsw"].append(w) for l in row["cast_tok"] for w in l if not w in stop_words]
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,writer_nsw,director_tok,writer_tok,cast_nsw,cast_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],"[the, bee]","[louis, h., tolhurst]",[],None,[],[],[]
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]","[mickey, makes, money]","[micky, dolenz]","[colin, bostock-smith]",None,None,"[metal, mickey, metal, mickey, michael, staint...",None
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]","[episode, dated, 6, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]","[episode, dated, 10, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]","[i, fuckin, ', miss, cory, and, trevor]","[mike, clattenburg]","[mike, clattenburg, mike, clattenburg, timm, h...",None,None,"[john, paul, tremblay, julian, robb, wells, ri...",None


Tokenize, remove stop words and then stem "plot summary"

In [57]:
test1['ps_stem'] = np.empty((len(test1), 0)).tolist()        
test1['ps_nsw'] = np.empty((len(test1), 0)).tolist()
test1["ps_tok"] = np.empty((len(test1), 0)).tolist()
for index,row in test1.iterrows():
    if pd.isnull(row["plot summary"])==False:
        test1.loc[index,"ps_tok"] = test1.loc[index,"ps_tok"].append(word_tokenize(row["plot summary"]))
        [test1.loc[index,"ps_nsw"].append(w) for l in row["ps_tok"] for w in l if not w in stop_words]
        for w in row["ps_nsw"]:
            test1.loc[index,"ps_stem"].append(str(ps.stem(w)))
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,writer_nsw,director_tok,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],"[the, bee]","[louis, h., tolhurst]",[],None,[],[],[],[],[],[]
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]","[mickey, makes, money]","[micky, dolenz]","[colin, bostock-smith]",None,None,"[metal, mickey, metal, mickey, michael, staint...",None,"['', mickey, gots, power, reproduce, money, or...","['', mickey, got, power, reproduc, money, orde...",None
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]","[episode, dated, 6, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[]
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]","[episode, dated, 10, march, 2007]","[paola, pol, balloussier, ary, coslov, gustavo...",[],None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[]
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]","[i, fuckin, ', miss, cory, and, trevor]","[mike, clattenburg]","[mike, clattenburg, mike, clattenburg, timm, h...",None,None,"[john, paul, tremblay, julian, robb, wells, ri...",None,"['', ricky, julian, bubbles, stealing, meat, g...","['', ricki, julian, bubbl, steal, meat, grocer...",None


Tokenize, remove stop words and then stem "plot keywords"

In [58]:
test1['pk_stem'] = np.empty((len(test1), 0)).tolist()        
test1['pk_nsw'] = np.empty((len(test1), 0)).tolist()
test1["pk_tok"] = np.empty((len(test1), 0)).tolist()
for index,row in test1.iterrows():
    if pd.isnull(row["plot keywords"])==False:
        test1.loc[index,"pk_tok"] = test1.loc[index,"pk_tok"].append(word_tokenize(row["plot keywords"]))
        [test1.loc[index,"pk_nsw"].append(w) for l in row["pk_tok"] for w in l if not w in stop_words]
        for w in row["pk_nsw"]:
            test1.loc[index,"pk_stem"].append(str(ps.stem(w)))
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,director_tok,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok,pk_stem,pk_nsw,pk_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],...,None,[],[],[],[],[],[],[],[],[]
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]",...,None,None,"[metal, mickey, metal, mickey, michael, staint...",None,"['', mickey, gots, power, reproduce, money, or...","['', mickey, got, power, reproduc, money, orde...",None,[],[],[]
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]",...,None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[],[],[],[]
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]",...,None,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[],[],[],[]
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]",...,None,None,"[john, paul, tremblay, julian, robb, wells, ri...",None,"['', ricky, julian, bubbles, stealing, meat, g...","['', ricki, julian, bubbl, steal, meat, grocer...",None,"[freezer, trapped-in-a-freez, pizza-delivery-b...","[freezer, trapped-in-a-freezer, pizza-delivery...",None


Tokenize "genre"

In [63]:
test1["genre_tok"] = [word_tokenize(i) for i in test1["genre"] if pd.isnull(i)==False]
test1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok,pk_stem,pk_nsw,pk_tok,genre_tok
12,1000066.0,the bee,documentary short,louis h. tolhurst,NaN,NaN,NaN,NaN,[bee],[bee],...,[],[],[],[],[],[],[],[],[],"[documentary, short]"
16,1000465.0,mickey makes money,family comedy sci-fi,micky dolenz,(metal mickey)-(metal mickey) (michael staint...,colin bostock-smith,"[u""mickey gots the power to reproduce money in...",NaN,"[mickey, makes, money]","[mickey, make, money]",...,None,"[metal, mickey, metal, mickey, michael, staint...",None,"['', mickey, gots, power, reproduce, money, or...","['', mickey, got, power, reproduc, money, orde...",None,[],[],[],"[family, comedy, sci-fi]"
18,1000495.0,episode dated 6 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 6, march, 2007]","[episod, date, 6, march, 2007]",...,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[],[],[],[],[drama]
19,1000499.0,episode dated 10 march 2007,drama,paola pol balloussier ary coslov gustavo fern...,(murilo benício)-(artur) (deborah secco)-(eli...,NaN,NaN,NaN,"[episode, dated, 10, march, 2007]","[episod, date, 10, march, 2007]",...,[],"[murilo, bencio, artur, deborah, secco, elizab...",None,[],[],[],[],[],[],[drama]
21,1000552.0,i fuckin' miss cory and trevor,comedy crime drama,mike clattenburg,(john paul tremblay)-(julian) (robb wells)-(r...,mike clattenburg mike clattenburg timm hanneb...,"[u""ricky julian and bubbles are stealing meat...",[ freezer' trapped-in-a-freezer' pizza-del...,"[fuckin, miss, cory, trevor]","[fuckin, miss, cori, trevor]",...,None,"[john, paul, tremblay, julian, robb, wells, ri...",None,"['', ricky, julian, bubbles, stealing, meat, g...","['', ricki, julian, bubbl, steal, meat, grocer...",None,"[freezer, trapped-in-a-freez, pizza-delivery-b...","[freezer, trapped-in-a-freezer, pizza-delivery...",None,"[comedy, crime, drama]"


In [64]:
test1.to_csv('test1.csv')